In [3]:
import torch
import numpy as np
from fastai.text.all import *

In [4]:
path = untar_data(URLs.MNIST_SAMPLE)

In [5]:
path.ls()

(#3) [Path('/Users/fabianjaeger/.fastai/data/mnist_sample/valid'),Path('/Users/fabianjaeger/.fastai/data/mnist_sample/labels.csv'),Path('/Users/fabianjaeger/.fastai/data/mnist_sample/train')]